# Analysis of payments and products:
1. Study the distribution of payment types and their impact on the success of deals.
2. Analyze the popularity and success of various products and training types.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [ ]:
deals = pd.read_pickle('deals_df.pkl')

# 1. Analysis of the distribution of payment types and their impact on the success of deals

In [ ]:
# Data Filtering: Excluding "Non applicable"
filtered_deals = deals[
    (deals['Payment Type'].astype(str) != 'Non applicable') &
    (deals['Payment Type'].astype(str) != 'Unknown')
]

# Calculating the number of unique processed deals for each payment type
total_payments = filtered_deals.groupby('Payment Type', observed=True)['Id'].nunique().reset_index(name='Total Deals')

# Calculating the number of successful deals for each payment type
successful_payments = filtered_deals[filtered_deals['Stage'] == 'Payment Done'].groupby('Payment Type', observed=True)['Id'].nunique().reset_index(name='Successful Deals')

# Merging data to calculate the conversion rate
payment_performance = pd.merge(total_payments, successful_payments, on='Payment Type', how='left')

# Filling missing values
payment_performance['Successful Deals'] = payment_performance['Successful Deals'].fillna(0)
payment_performance['Unsuccessful Deals'] = payment_performance['Total Deals'] - payment_performance['Successful Deals']

# Calculating the conversion rate
payment_performance['Conversion Rate'] = (payment_performance['Successful Deals'] / payment_performance['Total Deals'] * 100).round(1)

# Sorting data by total number of deals
payment_performance = payment_performance.sort_values('Total Deals', ascending=False)

# Creating charts
fig, axs = plt.subplots(1, 2, figsize=(14, 6))  # 1 row, 2 columns

# Chart 1: Number of Deals
axs[0].bar(payment_performance['Payment Type'], payment_performance['Total Deals'], color='brown', label='Total Number of Deals')
axs[0].set_title('Payment Type Analysis: Total Number of Deals')
axs[0].set_xlabel('Payment Type')
axs[0].set_ylabel('Number of Deals')
axs[0].tick_params(axis='x', rotation=45)

# Adding numbers above the bars
for i, v in enumerate(payment_performance['Total Deals']):
    axs[0].text(i, v + 0.5, f'{int(v)}', ha='center', va='bottom', fontsize=9)

# Chart 2: Conversion Rate
axs[1].bar(payment_performance['Payment Type'], payment_performance['Conversion Rate'], color='coral', label='Conversion Rate (%)')
axs[1].set_title('Payment Type Analysis: Conversion Rate')
axs[1].set_xlabel('Payment Type')
axs[1].set_ylabel('Conversion Rate (%)')
axs[1].tick_params(axis='x', rotation=45)

# Adding numbers above the bars
for i, v in enumerate(payment_performance['Conversion Rate']):
    axs[1].text(i, v + 0.5, f'{v:.1f} %', ha='center', va='bottom', fontsize=9)

# Adjusting the charts
plt.tight_layout()
plt.show()

# Formatting the table
styled_performance = payment_performance.style.format({
    'Total Deals': '{:,.0f}',
    'Successful Deals': '{:,.0f}',
    'Unsuccessful Deals': '{:,.0f}',
    'Conversion Rate': '{:.1f}%'
}).hide(axis='index').background_gradient(subset=['Conversion Rate'], cmap='YlOrRd')

display(styled_performance)

### Conclusion:
The most popular type of payment is "Recurring payments," with a total of 350 transactions, of which 250 were completed successfully, corresponding to a conversion of 71.4%. In turn, "One-time payment" also shows good results, although it lags behind recurring payments in all key metrics. The total number of transactions was 141, with 113 being successful, at a conversion of 80.1%. The lowest figures are observed in "Reservations" - only 5 transactions, but with a 20% conversion rate. Although this type of payment has a low conversion rate, its small number of transactions indicates the need to revisit the promotion and customer acquisition strategy in this area.

Overall, the data indicates that "One-time payment" is the most successful type of payment with a conversion of 80.1%.

# 2. Analysis of the popularity and success of various products and types of training

## Charts: Product Analysis: Total Number of Deals

In [ ]:
# Data Filtering: Excluding "Unknown"
filtered_deals = deals[deals['Product'].astype(str) != 'Unknown']

# Calculating the number of unique processed deals for each product
total_products = filtered_deals.groupby('Product', observed=True)['Id'].nunique().reset_index(name='Total Deals')

# Calculating the number of successful deals for each product
successful_products = filtered_deals[filtered_deals['Stage'] == 'Payment Done'].groupby('Product', observed=True)['Id'].nunique().reset_index(name='Successful Deals')

# Merging data to calculate the conversion rate
product_performance = pd.merge(total_products, successful_products, on='Product', how='left')
product_performance['Successful Deals'] = product_performance['Successful Deals'].fillna(0)
product_performance['Conversion Rate'] = (product_performance['Successful Deals'] / product_performance['Total Deals'] * 100).round(1)

# Sorting data by total number of deals
product_performance = product_performance.sort_values('Total Deals', ascending=False)

# Creating charts
fig, axs = plt.subplots(1, 2, figsize=(14, 6))  # 1 row, 2 columns

# Chart 1: Number of Deals
bars = axs[0].bar(product_performance['Product'], product_performance['Total Deals'], color='red', label='Total Number of Deals')
axs[0].set_title('Product Analysis: Total Number of Deals')
axs[0].set_xlabel('Product')
axs[0].set_ylabel('Number of Deals')
axs[0].tick_params(axis='x', rotation=45)

# Adding numbers above the bars
for bar in bars:
    height = bar.get_height()
    axs[0].text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}', ha='center', va='bottom', fontsize=9)

# Chart 2: Conversion Rate
bars_conversion = axs[1].bar(product_performance['Product'], product_performance['Conversion Rate'], color='orange', label='Conversion Rate (%)')
axs[1].set_title('Product Analysis: Conversion Rate')
axs[1].set_xlabel('Product')
axs[1].set_ylabel('Conversion (%)')
axs[1].tick_params(axis='x', rotation=45)

# Adding numbers above the bars
for bar in bars_conversion:
    height = bar.get_height()
    axs[1].text(bar.get_x() + bar.get_width() / 2, height + 1, f'{height:.1f} %', ha='center', va='bottom', fontsize=9)

# Optimizing and displaying charts
plt.tight_layout()
plt.show()

# Formatting the table
styled_performance = product_performance.style.format({
    'Total Deals': '{:,.0f}',
    'Successful Deals': '{:,.0f}',
    'Conversion Rate': '{:.1f}%'
}).hide(axis='index').background_gradient(subset=['Conversion Rate'], cmap='YlOrRd')

display(styled_performance)

## The dependence of product choice on the type of training

In [ ]:
# Data Filtering: Excluding 'Unknown' in 'Product' and limiting 'Education Type' to 'Morning' and 'Evening'
filtered_deals = deals[(deals['Product'].astype(str) != 'Unknown') & (deals['Education Type'].isin(['Morning', 'Evening']))].copy()

# Removing 'Unknown' from category columns (in case they remain)
filtered_deals['Product'] = filtered_deals['Product'].cat.remove_categories(['Unknown'])
filtered_deals['Education Type'] = filtered_deals['Education Type'].cat.remove_categories(['Unknown'])

# Checking unique values for debugging
print("Unique products:", filtered_deals['Product'].cat.categories)
print("Unique education types:", filtered_deals['Education Type'].cat.categories)

# Grouping data by 'Education Type' and 'Product' with a count for the chart
grouped = filtered_deals.groupby(['Education Type', 'Product'], observed=True).size().unstack(fill_value=0)

# Creating the chart
plt.figure(figsize=(12, 8))
ax = sns.heatmap(
    grouped,
    annot=True,
    fmt='d',
    cmap='YlOrRd',
    annot_kws={"size": 10},
    cbar_kws={'label': 'Number of Deals'},
    linewidths=0.5,
    linecolor='white'
)

# Setting the title and labels
plt.title('Dependency of Product Choice on Education Type', pad=20, fontsize=14)
plt.xlabel('Product', fontsize=12)
plt.ylabel('Education Type', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Optimizing layout
plt.tight_layout()

# Displaying the chart
plt.show()

# Preparing the table with conversion and Total Sales
# Calculating the total number of deals
total_deals = filtered_deals.groupby(['Education Type', 'Product'], observed=True)['Id'].nunique().reset_index(name='Total Deals')

# Calculating successful deals
successful_deals = filtered_deals[filtered_deals['Stage'] == 'Payment Done'].groupby(['Education Type', 'Product'], observed=True)['Id'].nunique().reset_index(name='Successful Deals')

# Calculating Total Sales (sum of Offer Total Amount for successful deals)
total_sales = (filtered_deals[filtered_deals['Stage'] == 'Payment Done']
    .groupby(['Education Type', 'Product'], observed=True)['Offer Total Amount']
    .sum()
    .reset_index(name='Total Sales'))

# Merging data
performance_table = total_deals.merge(successful_deals, on=['Education Type', 'Product'], how='left')
performance_table = performance_table.merge(total_sales, on=['Education Type', 'Product'], how='left')
performance_table['Successful Deals'] = performance_table['Successful Deals'].fillna(0)
performance_table['Total Sales'] = performance_table['Total Sales'].fillna(0)
performance_table['Unsuccessful Deals'] = performance_table['Total Deals'] - performance_table['Successful Deals']
performance_table['Conversion Rate'] = (performance_table['Successful Deals'] / performance_table['Total Deals'] * 100).round(1)

# Sorting by total number of deals
performance_table = performance_table.sort_values('Total Deals', ascending=False)

# Formatting the table
styled_table = performance_table.style.format({
    'Total Deals': '{:,.0f}',
    'Successful Deals': '{:,.0f}',
    'Unsuccessful Deals': '{:,.0f}',
    'Total Sales': '€{:,.0f}',  # Added format with euro symbol
    'Conversion Rate': '{:.1f}%'
}).hide(axis='index').background_gradient(subset=['Conversion Rate'], cmap='YlOrRd')

# Displaying the table
display(styled_table)

### Conclusion:
The analysis of the data in the table shows that the 'Morning' training type attracted the highest number of deals in the 'Digital Marketing' category (1,533 deals) with a total sales amount of €3,432,000. The morning courses in 'UX/UI Design' and 'Web Developer' also demonstrated 808 and 545 deals respectively, with sales amounts of €1,616,900 and €583,100.

At the same time, the 'Evening' training type demonstrated lower activity: 250 deals in 'Digital Marketing' with a sales amount of €408,800, 153 deals in 'UX/UI Design' with €217,500, and only one deal in 'Web Developer', which was unsuccessful.

The low number of deals in the evening courses, especially in the 'Web Developer' category (one deal only), indicates the need for a strategy review. Overall, morning courses are the main source of income and success, while evening courses require improvement. It may be worth strengthening the marketing of evening programs, enhancing their content, or revising the schedule to make them more appealing to clients.